In [154]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [155]:
import pandas as pd
import numpy as np
path = "/content/drive/MyDrive/S&P PIT Data(Sheet1).csv"
df = pd.read_csv(path)
df.head()


,Unnamed: 0,Return on Common Equity,Volatility 30 Day,Volatility 60 Day,Last Price,Moving Average 50 Day,Moving Average 200 Day,Current Market Cap,Trailing 12M Profit Margin,Trailing 12M Operating Margin,Total Debt to Total Equity,Price to Book Ratio,Price Earnings Ratio (P/E),Overridable Raw Beta,1 Month Total Return - Current,3 Month Total Return - Current,6 Month Total Return - Current
0,NaN,RETURN_COM_EQY,VOLATILITY_30D,VOLATILITY_60D,PX_LAST,MOV_AVG_50D,MOV_AVG_200D,CUR_MKT_CAP,TRAIL_12M_PROF_MARGIN,TRAIL_12M_OPER_MARGIN,TOT_DEBT_TO_TOT_EQY,PX_TO_BOOK_RATIO,PE_RATIO,BETA_RAW_OVERRIDABLE,CURRENT_TRR_1MO,CURRENT_TRR_3MO,CURRENT_TRR_6MO
1,A UN Equity,20.61871984,25.40712453,28.3392508,145.49,144.1791992,123.9143524,41181581310,18.75359816,21.28670121,49.75522919,6.109120734,27.41171465,0.967097194,-0.8518468,13.76815,26.14098
2,AAPL UW Equity,171.4224497,15.93944338,21.08221821,278.78,265.5939941,228.1499939,4.11935E+12,26.91506412,31.97079976,152.4107252,55.8567996,37.31994629,1.121690977,3.940481,17.30663,36.99901
3,ABBV UN Equity,NaN,29.14098002,27.23962734,226.08,228.1058044,204.6306458,3.9957E+11,4.003755617,15.15827242,2021.967895,NaN,26.20901689,0.242473156,3.15751,8.546165,21.01492
4,ABNB UW Equity,30.7755293,27.48934224,24.15574718,124.32,121.6735992,127.0525513,76525145666,22.02964079,22.64925061,27.27056586,8.908878027,29.64346289,1.255813029,2.8458,-0.2407343,-11.6041


In [156]:
# Map Bloomberg column names → our internal names
rename_map = {
    "Return on Common Equity": "roe_l2",
    "Volatility 30 Day": "vol30_l1",
    "Volatility 60 Day": "vol60_l1",
    "Last Price": "PX_LAST",
    "Moving Average 50 Day": "MOV_AVG_50D",
    "Moving Average 200 Day": "MOV_AVG_200D",
    "Current Market Cap": "mcap_raw",
    "Trailing 12M Profit Margin": "npm_l2",
    "Trailing 12M Operating Margin": "op_margin_l2",
    "Price Earnings Ratio (P/E)": "pe_raw",
    "Price to Book Ratio": "pb_raw",
    "Overridable Raw Beta": "beta_l1",
    "1 Month Total Return - Current": "mom_1m_l1",
    "3 Month Total Return - Current": "mom_3m_l1",
    "6 Month Total Return - Current": "mom_6m_l1",
    "Total Debt to Total Equity": "de_l2",
    # revps_growth_l2 and MA slopes ignored for now as you said
}

df = df.rename(columns=rename_map)


In [157]:
import numpy as np
import pandas as pd

# Drop the first row which contains string mnemonics instead of numerical data.
df = df.iloc[1:].copy()

# Rename 'Unnamed: 0' column to 'ticker' for clarity and consistency with later steps.
df = df.rename(columns={'Unnamed: 0': 'ticker'})

# Convert relevant columns to numeric, coercing errors to NaN
df["PX_LAST"] = pd.to_numeric(df["PX_LAST"], errors='coerce')
df["MOV_AVG_50D"] = pd.to_numeric(df["MOV_AVG_50D"], errors='coerce')
df["MOV_AVG_200D"] = pd.to_numeric(df["MOV_AVG_200D"], errors='coerce')
df["mcap_raw"] = pd.to_numeric(df["mcap_raw"], errors='coerce')
df["pe_raw"] = pd.to_numeric(df["pe_raw"], errors='coerce')
df["pb_raw"] = pd.to_numeric(df["pb_raw"], errors='coerce')
df["roe_l2"] = pd.to_numeric(df["roe_l2"], errors='coerce')
df["vol30_l1"] = pd.to_numeric(df["vol30_l1"], errors='coerce')
df["vol60_l1"] = pd.to_numeric(df["vol60_l1"], errors='coerce')
df["beta_l1"] = pd.to_numeric(df["beta_l1"], errors='coerce')
df["op_margin_l2"] = pd.to_numeric(df["op_margin_l2"], errors='coerce')
df["npm_l2"] = pd.to_numeric(df["npm_l2"], errors='coerce')
df["mom_1m_l1"] = pd.to_numeric(df["mom_1m_l1"], errors='coerce')
df["mom_3m_l1"] = pd.to_numeric(df["mom_3m_l1"], errors='coerce')
df["mom_6m_l1"] = pd.to_numeric(df["mom_6m_l1"], errors='coerce')
df["de_l2"] = pd.to_numeric(df["de_l2"], errors='coerce')

# Distance to 50-day MA: (PX_LAST − MOV_AVG_50D) / MOV_AVG_50D
df["dist50_l1"] = (df["PX_LAST"] - df["MOV_AVG_50D"]) / df["MOV_AVG_50D"]

# Distance to 200-day MA: (PX_LAST − MOV_AVG_200D) / MOV_AVG_200D
df["dist200_l1"] = (df["PX_LAST"] - df["MOV_AVG_200D"]) / df["MOV_AVG_200D"]

# Log market cap
df["log_mcap_l1"] = np.log(df["mcap_raw"].replace(0, np.nan))

# Earnings yield = 1 / PE
df["earn_yield_l2"] = 1 / df["pe_raw"].replace(0, np.nan)

# Book-to-price = 1 / PB
df["bp_l2"] = 1 / df["pb_raw"].replace(0, np.nan)


In [158]:
import pandas as pd
import numpy as np

# CHANGE THIS PATH in Colab to your Drive path
ma_path = "/content/drive/MyDrive/MA data(Sheet1).csv"

ma_raw = pd.read_csv(ma_path)

# Row 2: tickers (e.g. "A UN Equity", "AAPL UW Equity", etc.)
row_ticker = ma_raw.iloc[2]

# Row 4: Bloomberg field codes (MOV_AVG_200D / MOV_AVG_50D)
row_fieldcode = ma_raw.iloc[4]


In [159]:
ma_cols = {}
current_ticker = None

for j, val in enumerate(row_ticker):
    if j == 0:
        # first column is the "Dates" column
        continue

    # New ticker starts here
    if isinstance(val, str) and val.strip() != "":
        current_ticker = val.strip()
        if current_ticker not in ma_cols:
            ma_cols[current_ticker] = {}

    if current_ticker is None:
        continue

    field = row_fieldcode.iloc[j]
    if field == "MOV_AVG_200D":
        ma_cols[current_ticker]["ma200_col"] = j
    elif field == "MOV_AVG_50D":
        ma_cols[current_ticker]["ma50_col"] = j


In [160]:
# Data rows start from row index 5 (Dates + MA values)
data = ma_raw.iloc[5:].copy()

slopes = []

for ticker, cols in ma_cols.items():
    entry = {"ticker": ticker}

    # 50-day MA slope
    col_50 = cols.get("ma50_col")
    if col_50 is not None:
        s50 = pd.to_numeric(data.iloc[:, col_50], errors="coerce").dropna()
        if len(s50) >= 2:
            entry["ma50_slope_l1"] = (s50.iloc[-1] - s50.iloc[0]) / s50.iloc[0]
        else:
            entry["ma50_slope_l1"] = np.nan
    else:
        entry["ma50_slope_l1"] = np.nan

    # 200-day MA slope
    col_200 = cols.get("ma200_col")
    if col_200 is not None:
        s200 = pd.to_numeric(data.iloc[:, col_200], errors="coerce").dropna()
        if len(s200) >= 2:
            entry["ma200_slope_l1"] = (s200.iloc[-1] - s200.iloc[0]) / s200.iloc[0]
        else:
            entry["ma200_slope_l1"] = np.nan
    else:
        entry["ma200_slope_l1"] = np.nan

    slopes.append(entry)

slopes_df = pd.DataFrame(slopes).set_index("ticker")

slopes_df.head()


,ma50_slope_l1,ma200_slope_l1
ticker,,
A UN Equity,-0.000607,0.009926
AAPL UW Equity,0.014646,0.018338
ABBV UN Equity,0.014618,NaN
ABNB UW Equity,-0.007632,NaN
ABT UN Equity,0.001873,0.007937


In [161]:
# Ensure df_model is indexed by ticker
# If you still have a 'ticker' column instead:
# df_model = df_model.set_index("ticker")

# Use .update() to fill NaN values in df_model with the calculated slopes from slopes_df
df_model.update(slopes_df)

# Now, df_model contains the updated slope values. Create df_with_slopes for clarity
df_with_slopes = df_model.copy()

df_with_slopes.head()


,mom_1m_l1,mom_3m_l1,mom_6m_l1,dist50_l1,dist200_l1,vol30_l1,vol60_l1,ma50_slope_l1,ma200_slope_l1,log_mcap_l1,beta_l1,roe_l2,de_l2,op_margin_l2,npm_l2,earn_yield_l2,bp_l2,revps_growth_l2
ticker,,,,,,,,,,,,,,,,,,
A UN Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000607,0.009926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.029313
AAPL UW Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014646,0.018338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.024267
ABBV UN Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022555
ABNB UW Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042406
ABT UN Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001873,0.007937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016645


In [162]:
import pandas as pd
import numpy as np

# Path in Colab – change if needed
reveps_path = "/content/drive/MyDrive/reveps(Sheet1).csv"

rev_raw = pd.read_csv(reveps_path)

# Row 2: tickers
row_ticker = rev_raw.iloc[2]
# Row 4: field codes, should be TRAIL_12M_SALES_PER_SH
row_fieldcode = rev_raw.iloc[4]


In [163]:
rev_cols = {}
current_ticker = None

# Map each ticker → its TRAIL_12M_SALES_PER_SH column index
for j, val in enumerate(row_ticker):
    if j == 0:
        continue  # first col is Dates

    # New ticker starts here
    if isinstance(val, str) and val.strip() != "":
        current_ticker = val.strip()
        if current_ticker not in rev_cols:
            rev_cols[current_ticker] = None

    if current_ticker is None:
        continue

    field = row_fieldcode.iloc[j]
    if field == "TRAIL_12M_SALES_PER_SH":
        rev_cols[current_ticker] = j

# Data rows with actual values (dates + rev per share)
data = rev_raw.iloc[5:].copy()

rows = []
for ticker, col_idx in rev_cols.items():
    if col_idx is None:
        continue

    s = pd.to_numeric(data.iloc[:, col_idx], errors="coerce").dropna()
    if len(s) < 2:
        growth = np.nan
    else:
        prev = s.iloc[-2]
        curr = s.iloc[-1]
        growth = (curr - prev) / prev if prev not in [0, np.nan] else np.nan

    rows.append({"ticker": ticker, "revps_growth_l2": growth})

revps_growth_df = pd.DataFrame(rows).set_index("ticker")
revps_growth_df.head()


,revps_growth_l2
ticker,
A UN Equity,0.029313
AAPL UW Equity,0.024267
ABBV UN Equity,0.022555
ABNB UW Equity,0.042406
ABT UN Equity,0.016645


In [164]:
# ==== MERGE MAIN PIT DATA WITH SLOPES + REVPS GROWTH ====

# df: main PIT dataframe (already cleaned and with factors like mom, dist, vol, log_mcap, etc.)
# slopes_df: contains ma50_slope_l1 and ma200_slope_l1 indexed by ticker
# revps_growth_df: contains revps_growth_l2 indexed by ticker

# Ensure df has a ticker column; if it's currently the index, reset it
if df.index.name == "ticker" and "ticker" not in df.columns:
    df = df.reset_index()

# Merge slopes and revps growth onto df
df_all = df.merge(slopes_df, how="left", left_on="ticker", right_index=True)
df_all = df_all.merge(revps_growth_df, how="left", left_on="ticker", right_index=True)

# Define full factor list (including slopes + revps growth)
factor_cols = [
    "mom_1m_l1", "mom_3m_l1", "mom_6m_l1",
    "dist50_l1", "dist200_l1",
    "vol30_l1", "vol60_l1",
    "ma50_slope_l1", "ma200_slope_l1",
    "log_mcap_l1", "beta_l1",
    "roe_l2", "de_l2", "op_margin_l2", "npm_l2",
    "earn_yield_l2", "bp_l2", "revps_growth_l2"
]

# Make sure all factor columns exist
for c in factor_cols:
    if c not in df_all.columns:
        df_all[c] = np.nan

# Set ticker as index and keep only factor columns
df_all = df_all.set_index("ticker")
df_model = df_all[factor_cols]

df_model.head()


,mom_1m_l1,mom_3m_l1,mom_6m_l1,dist50_l1,dist200_l1,vol30_l1,vol60_l1,ma50_slope_l1,ma200_slope_l1,log_mcap_l1,beta_l1,roe_l2,de_l2,op_margin_l2,npm_l2,earn_yield_l2,bp_l2,revps_growth_l2
ticker,,,,,,,,,,,,,,,,,,
A UN Equity,-0.851847,13.768150,26.140980,0.009091,0.174117,25.407125,28.339251,-0.000607,0.009926,24.441257,0.967097,20.618720,49.755229,21.286701,18.753598,0.036481,0.163690,0.029313
AAPL UW Equity,3.940481,17.306630,36.999010,0.049647,0.221915,15.939443,21.082218,0.014646,0.018338,29.046716,1.121691,171.422450,152.410725,31.970800,26.915064,0.026795,0.017903,0.024267
ABBV UN Equity,3.157510,8.546165,21.014920,-0.008881,0.104820,29.140980,27.239627,0.014618,NaN,26.713655,0.242473,NaN,2021.967895,15.158272,4.003756,0.038155,NaN,0.022555
ABNB UW Equity,2.845800,-0.240734,-11.604100,0.021750,-0.021507,27.489342,24.155747,-0.007632,NaN,25.060885,1.255813,30.775529,27.270566,22.649251,22.029641,0.033734,0.112248,0.042406
ABT UN Equity,-1.012978,-4.954547,-5.514388,-0.027617,-0.044349,19.317347,18.014926,0.001873,0.007937,26.105455,0.279070,30.803306,31.888687,17.592318,31.879662,0.053107,0.234273,0.016645


In [165]:
# ==== WINSORIZE (1st / 99th PCT) + CROSS-SECTIONAL Z-SCORES ====

def winsorize_cross_section(df_in, lower=0.01, upper=0.99):
    df_w = df_in.copy()
    for col in df_w.columns:
        q_low = df_w[col].quantile(lower)
        q_high = df_w[col].quantile(upper)
        df_w[col] = df_w[col].clip(lower=q_low, upper=q_high)
    return df_w

def cross_section_z(df_in):
    return (df_in - df_in.mean()) / df_in.std(ddof=0)

df_w = winsorize_cross_section(df_model)
df_z = cross_section_z(df_w)

df_z.head()


,mom_1m_l1,mom_3m_l1,mom_6m_l1,dist50_l1,dist200_l1,vol30_l1,vol60_l1,ma50_slope_l1,ma200_slope_l1,log_mcap_l1,beta_l1,roe_l2,de_l2,op_margin_l2,npm_l2,earn_yield_l2,bp_l2,revps_growth_l2
ticker,,,,,,,,,,,,,,,,,,
A UN Equity,-0.499368,0.707138,0.651941,-0.066198,0.690473,-0.395664,-0.107812,-0.465326,0.212205,-0.110903,0.253417,-0.113558,-0.468732,0.160019,0.383699,-0.460591,-0.679099,0.290615
AAPL UW Equity,0.259434,0.933216,1.079766,0.523372,0.968981,-1.107863,-0.738309,0.739331,1.758391,3.498759,0.537005,4.713058,0.174021,0.946734,1.075101,-0.799362,-1.211543,0.102539
ABBV UN Equity,0.135461,0.373497,0.449966,-0.327469,0.286692,-0.114788,-0.203348,0.737121,NaN,1.898724,-1.075837,NaN,5.440778,-0.291243,-0.865841,-0.402037,NaN,0.038725
ABNB UW Equity,0.086105,-0.187910,-0.835280,0.117822,-0.449386,-0.239031,-0.471278,-1.020200,NaN,0.437074,0.783040,0.211520,-0.609515,0.260349,0.661230,-0.556657,-0.866976,0.778619
ABT UN Equity,-0.524881,-0.489083,-0.595335,-0.599834,-0.582481,-0.853763,-1.004798,-0.269490,-0.153489,1.360854,-1.008704,0.212409,-0.580599,-0.112014,1.495679,0.120947,-0.421313,-0.181544


In [166]:
from google.colab import files

df_z.to_csv('features_latest.csv', index=True)
files.download('features_latest.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>